In [ ]:
#| default_exp tui


# TUI - Terminal User Interface

Textual-based terminal user interface components for audio transcription.


In [ ]:
#| export
from time import monotonic
from enum import Enum
import asyncio

from textual import on
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, VerticalGroup, Grid, HorizontalGroup, CenterMiddle
from textual.widgets import Header, Footer, Static, Digits, Button, Label, TextArea

from tui_writer.live import LiveTranscriber
from tui_writer.ai import has_session, start_session, apply_instruction, current_transcript, reset_session

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        background: $surface;
    }
    Header {
        background: $primary;
    }

    StatusCard {
        width: auto;
        row-span: 2;
        border: solid $primary;
        border-title-align: center;
        height: 100%;
        padding: 1 0;
    }

    InfoCard {
        width: auto;
        border: solid #6b7280;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }

    TranscriptField {
        width: auto;
        border: solid $warning;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }
    TranscriptField TextArea {
        border: none;
    }

    #top-cards {
        layout: grid;
        grid-size: 4 2;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
        margin: 1;
        height: 40vh;
    }

    TextField {
        width: 100%;
        margin: 1;
        padding: 1;
        border: solid #8b5cf6;
        border-title-align: center;
        height: 50vh;
    }

    #status {
        content-align: center middle;
    }

    #status-time {
        border: double green;
        width: auto;
        padding: 0 1;
        margin: 1 0;
    }

    #media-controls {
        dock: bottom;
        align: center middle;
    }

    #media-controls Button {
        min-width: 20;
    }

    #pause,
    #resume,
    #stop {
        display: none;
    }

    .active #start {
        display: none;
    }
    .active #pause,
    .active #stop {
        display: block;
    }
    .active #status {
        color: $error;
    }

    .paused #start {
        display: none;
    }
    
    .paused #status {
        color: $warning;
    }

    .paused #resume,
    .paused #stop {
        display: block;
    }
"""

In [ ]:
#| export

class TimeDisplay(Digits):
    """Custom time display widget"""
    
    accumulated_time = 0
    time_elapsed = reactive(0)

    def on_mount(self):
        self.update_timer = self.set_interval(
            1 / 60, 
            self.update_time_elapsed,
            pause=True    
        )


    def update_time_elapsed(self):
        self.time_elapsed = self.accumulated_time + (monotonic() - self.start_time)

    def watch_time_elapsed(self):
        time = self.time_elapsed
        time, seconds = divmod(time, 60)
        hours, minutes = divmod(time, 60)
        time_string = f"{hours:02.0f}:{minutes:02.0f}:{seconds:05.2f}"
        self.update(time_string)

    def start(self):
        """Start keeping track of time elapsed"""
        self.start_time = monotonic()
        self.update_timer.resume()
    
    def pause(self):
        """Pause keeping track of time elapsed"""
        self.accumulated_time = self.time_elapsed
        self.update_timer.pause()

    def stop(self):
        """Stop/reset the time elapsed"""
        self.start_time = monotonic()
        self.accumulated_time = 0
        self.time_elapsed = 0
        self.update_timer.pause()

class StatusCard(Label):
    """Recording status card with controls"""
    status_text = reactive("● STANDBY")

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Recording Status"

    def start_transcript(self):
        self.add_class("active")
        self.status_text = "● RECORDING..."

    def pause_transcript(self):
        self.remove_class("active")
        self.add_class("paused")
        self.status_text = "⏸ PAUSED"

    def resume_transcript(self):
        self.remove_class("paused")
        self.add_class("active")
        self.status_text = "● RECORDING..."

    def stop_transcript(self):
        self.remove_class("paused")
        self.remove_class("active")
        self.status_text = "● STANDBY"

    def watch_status_text(self):
        """Update the status display when status_text changes"""
        if self.is_mounted:
            self.query_one("#status").update(self.status_text)

    def compose(self) -> ComposeResult:
        yield Static(self.status_text, id="status")
        with CenterMiddle():
            yield TimeDisplay("0:00:00", id="status-time")
        with HorizontalGroup(id="media-controls"):
            yield Button.error("● REC <space>", id="start")
            yield Button.success("⏸ PAUSE <space>", id="pause")
            yield Button.warning("▶ RESUME <space>", id="resume")
            yield Button("◼ STOP", id="stop")
    
class InfoCard(Label):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Info"

    def compose(self) -> ComposeResult:
        yield Static("Model: Whisper-Tiny")
        yield Static("Microphone: Default System Mic")

class TranscriptField(Label):

    transcript_block = reactive("", always_update=True)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Last Transcript-block"

    def compose(self) -> ComposeResult:
        yield TextArea("Change the name to Jens William", read_only=True, show_cursor=False)


class TextField(Label):

    content = reactive("")

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Text Field"

    def compose(self) -> ComposeResult:
        yield TextArea(
            "",
            read_only=True,
            show_line_numbers=True,
            show_cursor=False,
            id="main-textarea"
        )

    def watch_content(self) -> None:
        """This method is called whenever 'content' changes"""
        if self.is_mounted:
            textarea = self.query_one("#main-textarea")
            textarea.text = self.content
            textarea.refresh()  # This forces the TextArea to redraw!
            textarea.scroll_end()

## TranscriptionTUI

Main terminal user interface application for audio transcription.

The TUI uses Textual's [theme system](https://textual.textualize.io/guide/design/) with built-in color variables like `$primary`, `$error`, `$success`, etc. 

**Available built-in themes:**
- `textual-dark` (default)
- `textual-light`
- `nord`
- `gruvbox`
- `tokyo-night`
- `solarized-light`

To change the theme, modify the `self.theme` assignment in `__init__`.


In [ ]:
#| export

class TranscriptionTUI(App):

    TITLE = "TUI Writer"
    SUB_TITLE = "Design Template"
    CSS = TEXTUAL_CSS

    BINDINGS = [
        ("q", "quit", "Quit"),
        ("space", "toggle_status", "Start/Pause/Resume Recording"),
        ("l", "list_models", "List Models")
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.transcribe_task: asyncio.Task | None = None
        self.transcriber: LiveTranscriber | None = None

    def compose(self) -> ComposeResult:
        header = Header(show_clock=True)
        header.tall = True
        yield header

        with Container(id="top-cards"):
            yield StatusCard()
            yield InfoCard()  # Add an Info widget here
            yield TranscriptField()
        yield TextField()
        yield Footer()

    # Action methods triggered by key bindings, refers to button-methods
    async def action_toggle_status(self) -> None:
        running = bool(self.transcribe_task and not self.transcribe_task.done())
        await (self.stop_recording() if running else self.start_recording())
    
    async def on_button_pressed(self, event: Button.Pressed) -> None:
        if event.button.id == "start":
            await self.start_recording()
        elif event.button.id == "stop":
            await self.stop_recording()


    async def start_recording(self) -> None:
        if self.transcribe_task and not self.transcribe_task.done():
            return  # already running

        self.transcriber = LiveTranscriber(
            model_id="tiny",
            language="en",
            on_transcript=self.on_transcript_chunk,
            vad_threshold=0.5,
            min_speech_duration_ms=250,
            min_silence_duration_ms=500,
        )
        self.transcribe_task = asyncio.create_task(self.transcriber.start())  # runs until stopped

        self.query_one(StatusCard).start_transcript()
        self.query_one(TimeDisplay).start()

    def on_transcript_chunk(self, text: str) -> None:
        text = (text or "").strip()
        if not text:
            return
        self.query_one(TextField).content += text

    async def stop_recording(self) -> None:
        if not self.transcriber:
            return
        self.transcriber.stop()            # your class should flip its internal loop flag
        await asyncio.sleep(0.3)           # let it unwind any final callbacks

        if self.transcribe_task:
            try:
                self.transcribe_task.cancel()
                await self.transcribe_task
            except asyncio.CancelledError:
                pass

        self.transcriber = None
        self.transcribe_task = None

        self.query_one(StatusCard).stop_transcript()
        self.query_one(TimeDisplay).stop()

    async def on_unmount(self) -> None:
        await self.stop_recording()